In [4]:
import os
import sys

diretorio_atual = os.getcwd()
print(f"Logado no diretório: {diretorio_atual}")
sys.path.append('/home/jovyan/work')

Logado no diretório: /home/jovyan/work


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from datetime import datetime, timedelta

# Leitura dos dados originais
dados = pd.read_csv('./data/bank-additional/bank-additional-full.csv', sep=';')

# Variáveis numéricas e categóricas
variaveis_numericas = ['age', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
variaveis_categoricas = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

# Separando a variável alvo
y = dados['y']
X = dados.drop(columns=['y'])

# Gerador de dados sintéticos
def gerar_dados_sinteticos(X, y, n_samples=90000):
    dados_sinteticos = pd.DataFrame(columns=X.columns)
    
    for col in X.columns:
        if X[col].dtype == 'object':
            dados_sinteticos[col] = np.random.choice(X[col].unique(), size=n_samples, replace=True)
        else:
            dados_sinteticos[col] = np.random.normal(X[col].mean(), X[col].std(), size=n_samples)
    
    y_sintetico = np.random.choice(y.unique(), size=n_samples, replace=True)
    
    return dados_sinteticos, y_sintetico


def salvar_dados_sinteticos_com_data(safra, n_samples, alterar_distribuicao=False):
    # Gerando dados sintéticos
    X_sintetico, y_sintetico = gerar_dados_sinteticos(X, y, n_samples)
    
    # Alterar a distribuição das variáveis para algumas safras
    if alterar_distribuicao:
        for col in variaveis_numericas:
            X_sintetico[col] = X_sintetico[col] * np.random.uniform(0.5, 1.5, size=n_samples)
        for col in variaveis_categoricas:
            X_sintetico[col] = np.random.choice(X[col].unique(), size=n_samples, replace=True)
    
    # Adicionando a data ao DataFrame
    X_sintetico['data'] = safra.strftime('%Y-%m-%d')
    X_sintetico['y'] = y_sintetico
    
    # Salvando os dados sintéticos em CSV
    X_sintetico.to_csv(f'./data/Monitoramento/{safra.strftime("%Y_%m_%d")}_bank_markting.csv', index=False)

# Inicializando a data
safra = datetime(2020, 11, 25)
i = 0
while i < 30:
    if i % 3 == 0:  # Adiciona ruído a cada terceira safra
        salvar_dados_sinteticos_com_data(safra, n_samples=90000, alterar_distribuicao=True)
    else:
        salvar_dados_sinteticos_com_data(safra, n_samples=90000)
    safra = safra + timedelta(days=7)
    i += 1
